In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import Subset
import torch.nn.functional as F
import numpy as np

from cpmpy import *

## Saving a classifier

In this notebook, we will use the classifier that you built in p1.

Hence, first go to that notebook and _export_ the classifier you built there, by adding the following code in that notebook:


In [ ]:
#torch.save(model.state_dict(), "myNet")

## Loading a pre-trained classifier

Now, we can load that pre-trained classifier in this notebook as follows:

In [ ]:
def MLP():
    return nn.Sequential(nn.Flatten(), # Flatten MNIST images into a 784 long vector
                         nn.Linear(28*28, 120),
                         nn.ReLU(),
                         nn.Linear(120, 84), 
                         nn.ReLU(),
                         nn.Linear(84, 10),
                         nn.LogSoftmax(dim=1))



class LeNet(nn.Module):
    def __init__(self, calibrated=False):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5, 1, padding=2)
        self.conv2 = nn.Conv2d(6, 16, 5, 1)
        self.fc1 = nn.Linear(5*5*16, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84,10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 5*5*16) 
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

def load_clf(clf_classname, path):
    net = clf_classname()
    state_dict = torch.load(path, map_location=lambda storage, loc: storage)
    net.load_state_dict(state_dict)
    return net

model = load_clf(LeNet, 'myNet') # change arguments accordingly
model

We also have to load in the data again:

In [ ]:
# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])

# Download and load the training data
trainset = datasets.MNIST('.', download=False, train=True, transform=transform)
testset = datasets.MNIST('.', download=False, train=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=True)

## Recap: solving a sudoku based on the predictions

In the following, we repeat the code of the previous notebook for sampling a sudoku and getting predictions.

We also included example _ortools_ code that solves the sudoku problem _(requires to install ortools, e.g. conda install ortools)_

In [ ]:
# sudoku's, from http://hakank.org/minizinc/sudoku_problems2/index.html

sudoku_p0 = torch.IntTensor([[0,0,0, 2,0,5, 0,0,0],
                             [0,9,0, 0,0,0, 7,3,0],
                             [0,0,2, 0,0,9, 0,6,0],
                             [2,0,0, 0,0,0, 4,0,9],
                             [0,0,0, 0,7,0, 0,0,0],
                             [6,0,9, 0,0,0, 0,0,1],
                             [0,8,0, 4,0,0, 1,0,0],
                             [0,6,3, 0,0,0, 0,8,0],
                             [0,0,0, 6,0,8, 0,0,0]])

# sample a dataset index with that value/label
def sample_by_label(labels, value):
    # primitive but it works...
    idxs = torch.randperm(len(labels))
    for idx in idxs:
        if labels[idx] == value:
            return idx
# sample a dataset index for each non-zero number
def sample_visual_sudoku(sudoku_p, loader):
    for (images, labels) in loader: # sample one batch
        nonzero = sudoku_p > 0
        vizsudoku = torch.zeros((9,9,1,28,28), dtype=images.dtype)
        idxs = torch.LongTensor([sample_by_label(labels, value) for value in sudoku_p[nonzero]])
        vizsudoku[nonzero] = images[idxs]
        return vizsudoku
def show_grid_img(images):
    dim = 9
    figure = plt.figure()
    num_of_images = dim*dim
    for index in range(num_of_images):
        plt.subplot(dim, dim, index+1)
        plt.axis('off')
        plt.imshow(images[index].numpy().squeeze(), cmap='gray_r')
def predict_sudoku(model, vizsudoku):
    nonzero = (vizsudoku.reshape([-1,28,28]).sum(dim=(1,2)) != 0).reshape(9,9)
    predsudoku = torch.zeros((9,9), dtype=torch.long)
    with torch.no_grad():
        images = vizsudoku[nonzero]
        logprobs = model(images)
        preds = torch.argmax(logprobs, dim=1)
        predsudoku[nonzero] = preds
    return predsudoku
        
vizsudoku = sample_visual_sudoku(sudoku_p0, testloader)
show_grid_img(vizsudoku.reshape(-1,28,28))

## Preparing the solving code
In the subsequent, we can no longer assume that some values are given. Instead of constraining the non-empty values, we will add an objective function over the predicted probabilities.


__Task: Prepare your CPMpy model for this by splitting your `solve_sudoku()` of the previous notebook, into:__
    - `model_sudoku()` returns a CPMpy `Model` with all constraints except the 'value' constraints
    - `solve_sudoku()` that adds to that model the constraints on the values and solves it


In [ ]:
def model_sudoku(shape=(9,9)):
    maxv = max(shape)
    puzzle = IntVar(1,maxv, shape=shape)
    
    # ...
    
    m = Model()
    return (m, puzzle)

def solve_sudoku(grid, empty_val=0):
    """
        Solve sudoku with given grid 'grid', where 'empty_val' is used to denote empty cells.
        returns (Bool, Array) with:
            - Bool: whether a satisfying solution was found
            - Array: if Bool = True, the filled-in grid.
    """
    (m,puzzle) = model_sudoku()
    
    # Constraints on values (cells that are not empty)
    # ...
    
    if m.solve():
        return (True, puzzle.value())
    else:
        return (False, None)

# try solving the 'true' sudoku, which has a unique solution
solve_sudoku(sudoku_p0.numpy())

## Finding the maximum likelihood solution

As errors in the output may lead to infeasible sudoku's, we are going to want to find the _maximum likelihood_ solution.

First, we read and store the prediction probabilities instead of the predictions. We obtain a 9x9x10 tensor (last dimension = probabilities of digit 0..9)


In [ ]:
# get probabilities of predictions
def predict_proba_sudoku(model, vizsudoku):
    # reshape from 9x9x1x28x28 to 81x1x28x28
    pred = model(vizsudoku.flatten(0,1))
    # our NN return 81 probabilistic vector: an 81x10 matrix
    return pred.reshape(9,9,10).detach() # reshape as 9x9x10 tensor for easier visualisation

logprobs = predict_proba_sudoku(model, vizsudoku)

## Maximum likelihood estimation with standard CP solver

We need to turn the _satisfaction_ problem of sudoku into an _optimisation_ problem, where we optimize for maximum log likelihood.

This means adding the objective function: a weighted sum of the decision variables, with as weight the log-probability of that decision variable being equal to the corresponding predicted value.

E.g. $\sum_i \sum_j \sum_c -log(prob[i,j,c])*[V[i,j] == c]$ over the given digits

__Task: adapt the sudoku code to find the maximum likelihood visual sudoku solution!__

Note that the only thing that changes is adding the objective, so you can reuse `sudoku_model()` for the constraints part!

We assume that cells containing given clues are available through a $is\_given$ boolean matrix.

In [ ]:
def solve_vizsudoku(logprobs, is_given):
    # the constraint model
    csp, puzzle = model_sudoku(shape=(9,9))
    
    # 1) convert logprobs to positive integers (with scaling factor)
    logprobs = (-logprobs*10000).type(torch.int).numpy()

    # 2) build the list of terms to sum in cost function.
    # for every cell that is given, for v in 0..9: sum(logprobs[i,j,v]*(puzzle[i,j] == v))
    raise Exception("TODO!")
    #obj = sum( ... )
    
    # 3) minimize the cost
    m = Model(csp.constraints, minimize=obj)

    if m.solve():
        return (True, puzzle.value())
    else:
        return (False, None)


vsudoku = sample_visual_sudoku(sudoku_p0, testloader)
logprobs = predict_proba_sudoku(model, vsudoku)
is_given = (sudoku_p0 > 0)
(sat, psol) = solve_vizsudoku(logprobs, is_given)
print("Actual:\n", solve_sudoku(sudoku_p0.numpy())[1])

# let's show the other ones too...
preds = predict_sudoku(model, vsudoku).numpy()
print(f"Predictions solvable: {solve_sudoku(preds)[0]}, preds:\n",preds)
print("Max likelihood:\n", psol)

# Visualizing prediction error

Our trained Neural network classifies an image correctly if it assigns the highest score to the true label. Thus, we can assess the accuracy of the model by comparing maximum likelihood labels against labels in the numerical instance. 

In [ ]:
##helper function to plot and compare solution found with hybrid approach
def plot_vs(visualsudoku, output, is_given, ml_digits, solution):
    n = 9
    fig, axes = plt.subplots(n, n, figsize=(1.5*n,2*n))

    for i in range(n*n):
        ax = axes[i//n, i%n]
        # sample image wrt solver output
        img = torch.zeros(28,28).float()
        c = 'gray'
        if is_given.reshape(-1)[i]:
            img = visualsudoku.view(-1, 28,28)[i].squeeze()
            # wrong given -> red
            # given fixed by cp -> green
            c = 'gray' if output.reshape(-1)[i] == ml_digits.reshape(-1)[i] else 'summer'

        c = 'autumn' if is_given.reshape(-1)[i] and output.reshape(-1)[i] != solution.reshape(-1)[i] else c

        if c == 'summer':
            ax.set_title('ML label: {}\nsolver label: {}'.format(ml_digits.reshape(-1)[i], output.reshape(-1)[i]))
        elif c == 'autumn':
            ax.set_title('solver label: {}\nTrue label: {}'.format(output.reshape(-1)[i], solution.reshape(-1)[i]))

        ax.imshow(img, cmap=c)
        ax.set_axis_off()
plot_vs(vsudoku, psol, is_given, ml_digits, sudoku_p0)

# Higher Order knowledge exploitation

As an optimisation problem, a visual sudoku instance may have many feasible solutions. 

However, any valid sudoku puzzle only admits a unique solution for a set of givens. To improve the efficiency of our hybrid approach,  we can actually exploit this uniqueness property in the following manner:

- When solver finds optimal solution, add the resulting assignment as a no-good and try to solve again
- if any feasible solution is found this way, previous assignment for given cells does not lead to unique solution

Iterate these steps until the uniqueness property is satisfied. In practice, we may need to loop several times depending on the accuracy of the classifier.




In [ ]:
## Helpers 
def has_unique_solution(solution, is_given):
    """
    Check whether values in given cells from provided solution lead to a unique sudoku solution. 
    """
    sol = solution.astype(np.int)
    csp, board = model_sudoku((9,9))
    # Enforce the solved predictions of the givens
    csp += [board[is_given] == sol[is_given]]
    
    # Forbid the solved predictions of the others
    csp += all(board != sol)
    
    return not csp.solve()


def add_nogood(model, board, solution, is_given):
    """
    Forbid current assignement of value in given cells, from provided solution
    """
    sol = solution.astype(np.int)
    # nogood: built-in forbidden assignement constraint
    model.constraints += [board[is_given] != solution[is_given]]
    
    return model


## The actual function
def solve_vizsudoku_hybrid2(probs, is_given):
    (sat, solution) = solve_vizsudoku(probs, is_given)
    
    # Write a loop repeating following steps:
    # while ´solution´ is not unique:
    #   add nogood to the vizsudoku model
    #   solve again
    
    # Tip: you want access to the model created in `solve_sudoku` for this
            
    return False, None

Finally, we visualize the output 

In [ ]:
(sat, psol) = solve_vizsudoku_hybrid2(logprobs, is_given)

if sat:
    plot_vs(vsudoku, psol, is_given, ml_digits, sudoku_p0)